In [ ]:
import gc
import ctypes

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.decomposition import FastICA
import xgboost as xgb

from sklearn.feature_selection import SelectFromModel

from multiprocessing import Process, Manager

from sklearn.cluster import KMeans

def set_Data(data):
    ppmi = pd.read_csv('../../datasets/preprocessed/trans_processed_PPMI_data.csv')
    ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
    ppmi.set_index('Sentrix_position', inplace=True)
    ppmi = ppmi.transpose()

    encoder = LabelEncoder()
    label = encoder.fit_transform(ppmi['Category'])

    tr = ppmi.drop(['Category'], axis=1)
    X = tr.values
    y = label
    print(X.shape)
    print(y.shape)

#     print("StratifiedSampling check")
#     split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#     split.get_n_splits(X, y)

#     for train_index, test_index in split.split(X, y):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, data['y_test'] = y[train_index], y[test_index]

    print("Oversampling check")
    oversampler = SMOTE(random_state=42)
    X_sampled, data['y_sampled'] = oversampler.fit_resample(X, y)
    print(X_sampled.shape)
    print("Scaling check")
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_sampled)
    data['X_scaled_1'] = X_scaled[:300].reshape((1, -1))
    data['X_scaled_2'] = X_scaled[300:].reshape((1, -1))
    
    print("Returning check")

manager = Manager()
data = manager.dict()

print("CHECKPOINT1")
#     p = Process(target=set_Data, args=(X_train_scaled, X_test_scaled, y_train_sampled, y_test,))
p = Process(target=set_Data, args=(data,))
print("CHECKPOINT2")
p.start()
print("CHECKPOINT3")
p.join()

y_sampled = data['y_sampled']
X_scaled = np.append(data['X_scaled_1'], data['X_scaled_2']).reshape(618, 747668)

# print ("Shape of final train and test sets:", X_train_scaled.shape, X_test_scaled.shape)    

CHECKPOINT1
CHECKPOINT2
CHECKPOINT3


/home/rz296/miniconda3/envs/partII/lib/python3.6/multiprocessing/popen_fork.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,25

(436, 747668)
(436,)
Oversampling check


In [ ]:
## clustering
X_for_clustering = X_scaled.transpose()
print("Shape of X for clustering:",  X_for_clustering.shape)
### Need to tune n_clusters
ks =  [3]
cluster_reduced_dict={}
for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=0, n_jobs=k, verbose=1).fit(X_for_clustering)
    clusters = kmeans.labels_
    for n in range(k):
        print("Index of features in cluster", n+1, "is:")
        i = np.where(clusters == n)
        print(i)
        print("Number of features in cluster",n+1, "=", i[0].size)
        X_cluster = X_for_clustering[i].transpose()
        print("Shape of X_train_cluster",n+1,"is", X_cluster.shape)
        umap = UMAP(n_components=4)
        X_umap = umap.fit_transform(X_cluster)
        cluster_reduced_dict[n] = X_umap
        
        print("Cluters dict:")
        print(cluster_reduced_dict)
        temp = np.concatenate((cluster_reduced_dict[0], cluster_reduced_dict[1]), axis=1)
        X_reduced =  np.concatenate((temp, cluster_reduced_dict[2]), axis=1)
        print("Final shape of input data after clustering:", X_train.shape)

In [ ]:
### Split training and testing sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
split.get_n_splits(X_reduced, y_sampled)

for train_index, test_index in split.split(X_reduced, y_sampled):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test= y[train_index], y[test_index]

In [ ]:
### Train on the concatenated clusters
clf = xgb.XGBClassifier(
            objective='binary:logistic', 
            seed=42, 
            tree_method='gpu_hist',
            learning_rate=0.3,
            subsample=0.7,
            gpu_id=1,
            colsample_bytree=0.5,
            n_estimators=10,
            max_depth=4
        )

clf.fit(X_train, y_train_sampled)

y_pred_tr = clf.predict(X_train)
y_pred_te = clf.predict(X_test_scaled)
cm_tr = confusion_matrix(y_train_sampled, y_pred_tr)
cm_te = confusion_matrix(y_test, y_pred_te)
print("Confusion matrix of clustered + reduced training set using UMAP+XGBoost:")
print(cm_tr)
print("Confusion matrix of clustered + reduced testing set using UMAP+XGBoost:")
print(cm_te)
print("precision of testing set:", precision_score(y_test, y_pred_te))
